Emoji Prediction by Tony Ho

In [7]:
import pandas as pd
import tweepy
import emoji
import data_process
import sys

Twitter API account keys go here

In [8]:
consumer_key = ''
consumer_secret = ''

def twitter_Access():

    client = tweepy.Client(bearer_token='')

    return client

extract = twitter_Access()

Build queries for tweets with the top 9 emojis (recycle emoji times out too early, not enough tweets, so it is omitted).
Filter out non-English tweets, tweets with media, and retweets.

In [9]:
emoji_list = ['joy',
            'heart',
            'sob',
            'heart_eyes',
            #'recycle',
            'hearts',
            'blush',
            'two_hearts',
            'kissing_heart',
            'pensive',
            ]

emoji_list_icons = []

for emj in emoji_list:
    emoji_list_icons.append(emoji.emojize(":" + emj + ":",use_aliases=True))

queries = []

for emj in emoji_list_icons:
    queries.append("(" + emj + ") lang:en -is:retweet -has:media")

print(queries[0])

(😂) lang:en -is:retweet -has:media


Make requests to twitter, wait for 16 minutes of rate limitations imposed, then repeat for the next emoji.

In [13]:
import time

for i in range(len(emoji_list)):
    
    filename = "data/" + emoji_list[i] + "_no_media.txt"
    query_string = queries[i]

    with open(filename, 'a+') as f:

        nxt_tkn = None
        for i in range(1000):
            try:
                tweet_data = extract.search_recent_tweets(query=query_string, max_results=100, tweet_fields=['lang'], next_token=nxt_tkn)
                nxt_tkn = tweet_data.meta.get('next_token')
                data_process.continuous_tweet_processing(tweet_data, f) #replacing usernames and links with custom tokens
            except:
                print("Exception occured, likely over the limit")
                time.sleep(60 * 16)
                break

            if nxt_tkn == None:
                print("No next_token returned")
                break

Exception occured, likely over the limit
Exception occured, likely over the limit
Exception occured, likely over the limit
Exception occured, likely over the limit
Exception occured, likely over the limit
Exception occured, likely over the limit
Exception occured, likely over the limit
Exception occured, likely over the limit
Exception occured, likely over the limit


Since we are appending to an existing file, we have to remove duplicates since queries may overlap from the last time we made requests.

In [14]:
#get only unique lines
for emj in emoji_list:

    filename = "data/" + emj + "_no_media.txt"
    lines = []

    with open(filename, 'r') as f:
        for line in f:
            lines.append(line)

    lines = list(set(lines))

    with open(filename, 'w') as f:
        f.writelines(lines)

    print(len(lines))


133235
116071
131508
118838
119832
112968
121643
120123
129393


Process raw tweet by splitting the tweet into text segments, and identifying the correct emoji as the label. Save into a csv.

In [12]:
from data_process import tweet2csv

for i, emj in enumerate(emoji_list):
    filename = "data/" + emj

    with open(filename + "_no_media.txt", 'r') as f:
        lines = f.readlines()

        with open(filename + "_no_media.csv", 'w') as f_csv:
            f_csv.write("emoji,text\n")

            for line in lines:
                csv_lines = tweet2csv(line, emoji_list_icons[i])

                for csv_line in csv_lines:
                    if emoji.is_emoji(csv_line[0]): #check against weird characters
                        f_csv.write(csv_line)
                        
